In [1]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
pd.options.display.max_columns=1000
pd.options.display.max_rows=1000
pd.set_option('display.max_colwidth', -1)
sys.path.insert(0, "C:\\Users\\vberlia\\Documents\\machine_learning")
import itertools 

In [10]:
from machineLearning.dataSummary import DataSummary
from machineLearning.visualizations import Visualization
from machineLearning.ploty_visualization import PlotlyVisualization
import plotly.plotly as py
from machineLearning.missingValues import MissingValue
from machineLearning.models import Model
from machineLearning.modelInputs import KNNInputs
from machineLearning.pipelines import Pipelines
from machineLearning.featureCreation import CreateMeanLookupFeature
from machineLearning.featureCreation import CreateMedianLookupFeature
from machineLearning.featureCreation import CreateFrequencyLookupFeature
from machineLearning.featureCreation import CreateOneHotEncoding
from machineLearning.credit_prediction import classification_report
from machineLearning.featureCreation import CustomCutter
from machineLearning.credit_prediction import raw_features
from machineLearning.credit_prediction import create_features
from machineLearning.credit_prediction import prep_data
from machineLearning.featureCreation import IsMissingFeature
from sklearn import linear_model
from machineLearning.missingValues import CustomQuantitativeImputer
from machineLearning.misc import Misc
from machineLearning.missingValues import CustomEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import ggplot
from ggplot import *
import numpy as np

In [3]:
application_train=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/application_train.csv")
# application_newData=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/application_test.csv")
# bureau=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/bureau.csv")
# bureau_balance=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/bureau_balance.csv")
# # credit_card_balance=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/credit_card_balance.csv")
home_credit_col_desc=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/HomeCredit_columns_description.csv",encoding = "ISO-8859-1")
# intall_payment=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/installments_payments.csv")
# pos_cash=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/POS_CASH_balance.csv")
# prev_app=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/previous_application.csv")
# sample_submi=pd.read_csv("C:/Users/vberlia/Documents/data/credit_prediction/all/sample_submission.csv")

In [4]:
train_sample=application_train.sample(10000,random_state=0)
train_sample=train_sample.reset_index()
original_index=train_sample['index']
train_sample=train_sample.drop('index',axis=1)

X=train_sample.values
Y=train_sample['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)
X_train=pd.DataFrame(X_train)
X_train.columns=train_sample.columns
X_test=pd.DataFrame(X_test)
X_test.columns=train_sample.columns

In [27]:
raw_columns = raw_features.features.raw_columns

"""
Enter the create features function
and data preparer function
"""
final_pipeline=create_features.CreateFeatures.create_all_features1()
dataPreparer=prep_data.PrepData.dataPreparer1

"""
Fitting Logistic Regression: 
Setup the hyperparameters
"""

algorithm='LR'

class_weight=['balanced','None']
C=[0.001, 0.01, 0.1, 0.3,0.5,0.7,0.9,1,2,5,9,13,15,17,19,21,50,100,1000]
penalty=['l2']

# class_weight=['balanced']
# C=[21]
# penalty=['l2']

hyperparams=[class_weight,C,penalty]
combs=list(itertools.product(*hyperparams))

# Save the result in the file
result_file=""

In [28]:
# Model Function
def build_model(X_train,standardize_input=1,save_result=False,running_in_cdsw=False):
    for col in raw_columns:
        X_train[col]=X_train[col].astype('float64')

    for col in raw_columns:
        X_test[col]=X_test[col].astype('float64')

    # Data Preparer function -> should prepare data for all train, test and new data. Because
    # every data kind(train, test and new data) goes through the same preparation 
    # phase
    has_fitted_the_main_pipelines=0

    X_train['TARGET']=X_train['TARGET'].astype('int')
    X_test['TARGET']=X_test['TARGET'].astype('int')
    '''
    This should be underlying order
    '''
    X_train_prepared,has_fitted_the_main_pipelines=dataPreparer(X_train,has_fitted_the_main_pipelines,final_pipeline,'train')
    X_test_prepared,has_fitted_the_main_pipelines=dataPreparer(X_test,has_fitted_the_main_pipelines,final_pipeline,'test')
    if running_in_cdsw:
        new_data_prepared,has_fitted_the_main_pipelines=dataPreparer(application_newData,has_fitted_the_main_pipelines,final_pipeline,'new_data')

    col_list=X_train_prepared.columns
    # Apply scaling
    if standardize_input==1:
        standardiser= StandardScaler()
        X_train_target=X_train_prepared['TARGET']
        X_test_target=X_test_prepared['TARGET']

        standardiser.fit(X_train_prepared.iloc[:, :-1])
        X_train_prepared=pd.DataFrame(standardiser.transform(X_train_prepared.iloc[:, :-1]))
        X_train_prepared['TARGET']=X_train_target
        X_test_prepared=pd.DataFrame(standardiser.transform(X_test_prepared.iloc[:, :-1]))
        X_test_prepared['TARGET']=X_test_target
        if running_in_cdsw:
            new_data_prepared=pd.DataFrame(standardiser.transform(new_data_prepared))
            new_data_prepared.columns=col_list[:-1]

    X_train_prepared.columns=col_list
    X_test_prepared.columns=col_list
#     X_train_prepared.to_csv("C:/Users/vberlia/Documents/data/prep1.csv",index=None)
    

    """
    Fitting Logistic Regression: 
    """
    all_report=pd.DataFrame()
    for param in combs:

        model = linear_model.LogisticRegression(solver='sag',n_jobs=2)
        params = {"penalty":param[2], "C":param[1],"class_weight":param[0]}

        model.set_params(**params)
        model.fit(X_train_prepared.iloc[:, :-1], X_train_prepared.iloc[:, -1])
        accuracy_lr = model.score(X_test_prepared.iloc[:, :-1], X_test_prepared.iloc[:, -1])        

        prediction_result = pd.DataFrame()
        predictions = pd.DataFrame(model.predict_proba(X_test_prepared.iloc[:, :-1]))
        predictions.columns = ['class_' + str(item) for item in list(model.classes_)]
        predictions['true_label'] = X_test_prepared.iloc[:, -1]
        predictions['predicted_label'] = model.predict(X_test_prepared.iloc[:, :-1])
        predictions['model'] = 'LR'
        predictions['pipeline'] = 'no-pipeline'
        predictions['param'] = str(params)
        prediction_result = pd.concat([prediction_result, predictions])        
        report=classification_report.ClassificationReport.showClassificationReport(prediction_result)
        report.insert(0, 'algorithm', algorithm)
        report.insert(0, 'params', str(params))
        report.insert(0, 'model', model)
        all_report=pd.concat([all_report,report])
    if save_result:    
        all_report.to_csv(result_file)
    if running_in_cdsw:    
        return  new_data_prepared,all_report
    else:
        return all_report

In [29]:
all_report=build_model(X_train)

This function will generate the classification report for each pipelines best parameter fit classification report:

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


Model -> {'LR'}
Pipeline -> no-pipeline

Param -> {"{'penalty': 'l2', 'C': 0.001, 'class_weight': 'balanced'}"}

Confusion Matrix:
[[649 261]
 [ 29  61]]

Auc Score: 0.7556532356532357

Classification Report:
             precision    recall  f1-score   support

          0       0.96      0.71      0.82       910
          1       0.19      0.68      0.30        90

avg / total       0.89      0.71      0.77      1000



C:\Users\vberlia\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



This function will generate the classification report for each pipelines best parameter fit classification report:

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


Model -> {'LR'}
Pipeline -> no-pipeline

Param -> {"{'penalty': 'l2', 'C': 0.01, 'class_weight': 'balanced'}"}

Confusion Matrix:
[[632 278]
 [ 29  61]]

Auc Score: 0.7528449328449328

Classification Report:
             precision    recall  f1-score   support

          0       0.96      0.69      0.80       910
          1       0.18      0.68      0.28        90

avg / total       0.89      0.69      0.76      1000



C:\Users\vberlia\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



This function will generate the classification report for each pipelines best parameter fit classification report:

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


Model -> {'LR'}
Pipeline -> no-pipeline

Param -> {"{'penalty': 'l2', 'C': 0.1, 'class_weight': 'balanced'}"}

Confusion Matrix:
[[637 273]
 [ 31  59]]

Auc Score: 0.7486568986568988

Classification Report:
             precision    recall  f1-score   support

          0       0.95      0.70      0.81       910
          1       0.18      0.66      0.28        90

avg / total       0.88      0.70      0.76      1000



...

C:\Users\vberlia\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



This function will generate the classification report for each pipelines best parameter fit classification report:

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


Model -> {'LR'}
Pipeline -> no-pipeline

Param -> {"{'penalty': 'l2', 'C': 50, 'class_weight': 'None'}"}

Confusion Matrix:
[[907   3]
 [ 90   0]]

Auc Score: 0.751941391941392

Classification Report:
             precision    recall  f1-score   support

          0       0.91      1.00      0.95       910
          1       0.00      0.00      0.00        90

avg / total       0.83      0.91      0.87      1000



C:\Users\vberlia\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



This function will generate the classification report for each pipelines best parameter fit classification report:

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


Model -> {'LR'}
Pipeline -> no-pipeline

Param -> {"{'penalty': 'l2', 'C': 100, 'class_weight': 'None'}"}

Confusion Matrix:
[[908   2]
 [ 90   0]]

Auc Score: 0.7520390720390722

Classification Report:
             precision    recall  f1-score   support

          0       0.91      1.00      0.95       910
          1       0.00      0.00      0.00        90

avg / total       0.83      0.91      0.87      1000

This function will generate the classification report for each pipelines best parameter fit classification report:

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


Model -> {'LR'}
Pipeline -> no-pipeline

Param -> {"{'penalty': 'l2', 'C': 1000, 'class_weight': 'None'}"}

Conf

C:\Users\vberlia\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [31]:
all_report.shape

(38, 16)

In [30]:
all_report

,model,params,algorithm,AUC,class_0_precision,class_0_recall,class_0_f1_score,class_0_support,class_1_precision,class_1_recall,class_1_f1_score,class_1_support,total_precision,total_recall,total_f1_score,total_support
1,"LogisticRegression(C=0.001, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.001, 'class_weight': 'balanced'}",LR,0.755653,0.96,0.71,0.82,910,0.19,0.68,0.30,90,0.89,0.71,0.77,1000
1,"LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.01, 'class_weight': 'balanced'}",LR,0.752845,0.96,0.69,0.80,910,0.18,0.68,0.28,90,0.89,0.69,0.76,1000
1,"LogisticRegression(C=0.1, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.1, 'class_weight': 'balanced'}",LR,0.748657,0.95,0.70,0.81,910,0.18,0.66,0.28,90,0.88,0.70,0.76,1000
1,"LogisticRegression(C=0.3, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.3, 'class_weight': 'balanced'}",LR,0.747009,0.95,0.70,0.81,910,0.18,0.64,0.28,90,0.88,0.69,0.76,1000
1,"LogisticRegression(C=0.5, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.5, 'class_weight': 'balanced'}",LR,0.746777,0.95,0.70,0.81,910,0.18,0.64,0.28,90,0.88,0.69,0.76,1000
1,"LogisticRegression(C=0.7, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.7, 'class_weight': 'balanced'}",LR,0.746740,0.95,0.70,0.81,910,0.18,0.64,0.28,90,0.88,0.70,0.76,1000
1,"LogisticRegression(C=0.9, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 0.9, 'class_weight': 'balanced'}",LR,0.746764,0.95,0.70,0.81,910,0.18,0.64,0.28,90,0.88,0.69,0.76,1000
1,"LogisticRegression(C=1, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 1, 'class_weight': 'balanced'}",LR,0.746728,0.95,0.70,0.81,910,0.18,0.64,0.28,90,0.88,0.70,0.76,1000
1,"LogisticRegression(C=2, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 2, 'class_weight': 'balanced'}",LR,0.746606,0.95,0.70,0.81,910,0.17,0.64,0.27,90,0.88,0.69,0.76,1000
1,"LogisticRegression(C=5, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=2, penalty='l2', random_state=None,\n solver='sag', tol=0.0001, verbose=0, warm_start=False)","{'penalty': 'l2', 'C': 5, 'class_weight': 'balanced'}",LR,0.746642,0.95,0.70,0.81,910,0.17,0.64,0.27,90,0.88,0.69,0.76,1000
